In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 8.8 MB/s 


In [31]:
pip install spacy==3.2

     |████████████████████████████████| 6.0 MB 5.0 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.1
    Uninstalling spacy-3.2.1:
      Successfully uninstalled spacy-3.2.1


In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd 
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import collections

In [2]:
!python -m spacy download ru_core_news_sm

     |████████████████████████████████| 16.4 MB 22 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [3]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import spacy
from spacy.lang.ru import Russian
from spacy.lang.ru.examples import sentences 

In [5]:
import ru_core_news_sm

In [6]:
nlp = ru_core_news_sm.load()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Из reviews в таблицу информации о токенах (formated_reviews).

In [24]:
train_texts = pd.read_csv('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/Автобрея проект/Проект/train_split_reviews.txt',
                          delimiter='\t', names=['text_id','text'])
train_texts

,text_id,text
0,25709,"Были 1 февраля. Я второй раз, подруга - первый..."
1,10099,Были в Дитае на прошлой неделе с подругой. Заш...
2,1105,Один из моих любимых ресторанов. С самого откр...
3,3647,Остались очень довольны. Хотим поблагодарить ш...
4,37070,приехали к друзьям в Питер из Москвы на Новый ...
...,...,...
208,10645,Заезжали поужинать с подругой пару недель наза...
209,12341,"Ходила туда вчера. Интерьер необычный, привлек..."
210,34196,Бронировали столик в прошлые выходные (30.08.1...
211,17475,Была вчера в этом ресторане!Поехала туда не це...


In [25]:
train_asp = pd.read_csv('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/Автобрея проект/Проект/train_split_aspects.txt',
                        delimiter='\t', 
                        names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment'])
train_asp

,text_id,category,mention,start,end,sentiment
0,30808,Whole,ресторане,16,25,neutral
1,30808,Interior,первом этаже,43,55,neutral
2,30808,Whole,руководству ресторана,124,145,positive
3,30808,Service,обслуживающему персоналу,147,171,positive
4,30808,Service,сотрудникам,189,200,positive
...,...,...,...,...,...,...
3568,16630,Service,обслуживание,85,97,positive
3569,16630,Food,Еда,99,102,positive
3570,16630,Service,персоналу,244,253,positive
3571,16630,Whole,ресторан,294,302,positive


In [45]:
info = pd.DataFrame(columns=['text_id', 'sentence', 'word', 'lemma', 'POS', 'head', 'cat01', 'aspect', 'start',	'end', 'sentiment'])
info

,text_id,sentence,word,lemma,POS,head,cat01,aspect,start,end,sentiment


In [44]:
symbols = list('Привмиет, мир!')
symbols_inword = list('мир')
start = symbols.index(symbols_inword[0])
for i in range(len(symbols_inword)):
    if symbols.index(symbols_inword[i], start) > start+i:
        start = symbols.index(symbols_inword[i], start) - i
end = start+len(symbols_inword)
symbols, symbols_inword, start, end

(['П', 'р', 'и', 'в', 'м', 'и', 'е', 'т', ',', ' ', 'м', 'и', 'р', '!'],
 ['м', 'и', 'р'],
 10,
 13)

In [46]:
for i in range(len()):
    sent_tokens = sent_tokenize(train_texts['text'][i])
    for s_id in range(len(sent_tokens)):
        doc = nlp(sent_tokens[s_id])
        for token in doc:
            cat01 = 0
            asp = 0
            sentiment = '-'
            df_asp = train_asp.loc[train_asp['text_id'] == train_texts['text_id'][i]]
            for index, row in df_asp.iterrows():
                if row['mention'] == token.text:
                    cat01 = 1
                    asp = row['category']
                    start = row['start']
                    end = row['end']
                    sentiment = row['sentiment']
                else:
                    symbols = list(sent_tokens[s_id])
                    symbols_inword = list(token.text)

                    start = symbols.index(symbols_inword[0])
                    for i in range(len(symbols_inword)):
                        if symbols.index(symbols_inword[i], start) > start+i:
                            start = symbols.index(symbols_inword[i], start) - i
                    end = start+len(token.text)
            info.loc[len(info)+1] = [train_texts['text_id'][i], sent_tokens[s_id], token.text, token.lemma_, token.pos_, token.head, cat01, asp, start, end, sentiment]
info

,text_id,sentence,word,lemma,POS,head,cat01,aspect,start,end,sentiment
1,3647,Были 1 февраля.,Были,были,AUX,1,0,0,0,4,-
2,25709,Были 1 февраля.,1,1,ADJ,1,0,0,5,6,-
3,30201,Были 1 февраля.,февраля,февраль,NOUN,1,0,0,7,14,-
4,25709,Были 1 февраля.,.,.,PUNCT,1,0,0,14,15,-
5,25709,"Я второй раз, подруга - первый.",Я,я,PRON,раз,0,0,0,1,-
...,...,...,...,...,...,...,...,...,...,...,...
711,25709,"Все очень понравилось: кухня, интерьер,еда , д...",!,!,PUNCT,понравилось,0,0,228,229,-
712,25709,а так все замечательно,а,а,CCONJ,замечательно,0,0,0,1,-
713,1105,а так все замечательно,так,так,ADV,а,0,0,2,5,-
714,1105,а так все замечательно,все,все,PRON,а,0,0,6,9,-


In [100]:
info.to_csv('train_split_formated_reviews.csv')

# Из таблицы formated_reviews в таблицу с аспектами.

In [12]:
train_asp = pd.read_csv('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/Автобрея проект/Проект/train_formated_reviews.csv',
                        delimiter=',')
train_asp

,Unnamed: 0,text_id,sentence,word,lemma,POS,head,cat01,aspect,start,end
0,1,3976,"День 8-го марта прошёл, можно и итоги подвести.",День,день,NOUN,прошёл,0,0,-,-
1,2,3976,"День 8-го марта прошёл, можно и итоги подвести.",8-го,8-й,ADJ,День,0,0,-,-
2,3,3976,"День 8-го марта прошёл, можно и итоги подвести.",марта,март,NOUN,8-го,0,0,-,-
3,4,3976,"День 8-го марта прошёл, можно и итоги подвести.",прошёл,пройти,VERB,прошёл,0,0,-,-
4,5,3976,"День 8-го марта прошёл, можно и итоги подвести.",",",",",PUNCT,можно,0,0,-,-
...,...,...,...,...,...,...,...,...,...,...,...
47029,47030,16630,"Спасибо персоналу, ему удается создать ощущени...",),),PUNCT,Спасибо,0,0,-,-
47030,47031,16630,"Спасибо персоналу, ему удается создать ощущени...",),),PUNCT,Спасибо,0,0,-,-
47031,47032,16630,"Спасибо персоналу, ему удается создать ощущени...",),),PUNCT,Спасибо,0,0,-,-
47032,47033,16630,"Спасибо персоналу, ему удается создать ощущени...",),),PUNCT,Спасибо,0,0,-,-


In [20]:
df_aspects = pd.DataFrame(columns=['text_id', 'category', 'mention', 'start', 'end', 'sentiment'])

for index, row in train_asp[:100].iterrows():
  if row['aspect'] != '0':
      df_aspects.loc[len(df_aspects)+1] = [row['text_id'], row['aspect'], row['word'], row['start'], row['end'], 0]

In [21]:
df_aspects

,text_id,category,mention,start,end,sentiment
1,3976,Whole,ресторане,476,485,0
2,3976,Whole,ресторанах,198,208,0
3,3976,Whole,ресторане,476,485,0
4,3976,Service,администратор,322,335,0
5,3976,Whole,ресторан,1019,1027,0
6,3976,Whole,ресторане,476,485,0
7,3976,Food,Меню,564,568,0


In [23]:
df_aspects.to_csv('train_aspect_from_formated_reviews.csv', sep='\t')